# Data Exploration using SQL

In the below cells we use SQL for quering the database and trying to answer the questions set out.

The database used in SQLite and we are using sqlite3 package in Python to run the queries.

I have interacted with the databases using methods such as LIKE operator, Group By, Aggregation, Rank and Lag Function and working with date column to get the desired output.

In [34]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

import sqlite3
from sqlite3 import Error

In [35]:
conn = sqlite3.connect('airline.sqlite')

## Creating Tables in SQLite

In [30]:
def create_table(table_name,query,df):
    conn.execute(query)
    df.to_sql(table_name,conn,if_exists='replace',index=False)
    conn.commit()
    print("Success")

In [ ]:
airline_mod = pd.read_excel('D:/Others/dataset/airline/passenger_travel_data.xlsx')
airline_mod.drop(columns = ['Unnamed: 0'],inplace = True)
airline_mod.head()
#airline_mod.to_csv('D:/Others/dataset/airline/passenger_travel_data.csv')

table_name = 'passenger_travel_data'
query = f'Create table if not Exists {table_name} (data_dte date,usg_apt_id int,usg_apt text,usg_wac int,\
fg_apt_id int,fg_apt text,fg_wac int,airlineid int,carrier text,carriergroup int,scheduled int,charter int, total int)'
create_table(table_name,query,airline_mod)

In [ ]:
all_airport_codes = pd.read_excel('D:/Others/dataset/airline/airport_codes.xlsx')
all_airport_codes.drop(columns = ['Unnamed: 0'],inplace = True)
all_airport_codes.head()
#all_airport_codes.to_csv('D:/Others/dataset/airline/airport_codes.csv')

table_name = 'airport_codes'
query = f'Create table if not Exists {table_name} (city_name text,airport_code text,airport_name text,country_name text)'
create_table(table_name,query,all_airport_codes)

In [ ]:
airline_code = pd.read_excel('D:/Others/dataset/airline/airline_codes.xlsx')
airline_code.drop(columns = ['Unnamed: 0'],inplace = True)
airline_code.head()
#airline_code.to_csv('D:/Others/dataset/airline/airline_codes.csv')

table_name = 'airline_codes'
query = f'Create table if not Exists {table_name} (code int, airline_id text, airline_name text)'
create_table(table_name,query,airline_code)

In [ ]:
world_data = pd.read_csv('D:/Others/dataset/airline/continents-according-to-our-world-in-data.csv')
world_data.drop(columns = ['Code','Year'],inplace = True)
world_data.rename(columns = {'Entity':'country_name','Continent':'continent'}, inplace = True)
world_data.head()
#world_data.to_csv('D:/Others/dataset/airline/continents_data.csv')

table_name = 'world_data'
query = f'Create table if not Exists {table_name} (country_name text, continent text)'
create_table(table_name,query,world_data)

## Running SQL Queries 

### Finding travel details for the JFK airport

In [5]:
df = pd.read_sql('select * from passenger_travel_data where usg_apt = "JFK"',conn)
df

,data_dte,usg_apt_id,usg_apt,usg_wac,fg_apt_id,fg_apt,fg_wac,airlineid,carrier,carriergroup,scheduled,charter,total
0,2010-09-01 00:00:00,12478,JFK,22,15595,VIE,403,19873,OS,0,14636,0,14636
1,2007-07-01 00:00:00,12478,JFK,22,10872,CAI,591,19747,MS,0,16181,0,16181
2,2010-03-01 00:00:00,12478,JFK,22,13560,MXP,450,19790,DL,1,6851,0,6851
3,2008-10-01 00:00:00,12478,JFK,22,14717,SCL,324,19552,LA,0,7279,0,7279
4,2010-09-01 00:00:00,12478,JFK,22,10411,AUA,277,20409,B6,1,11564,0,11564
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67500,2022-06-01 00:00:00,12478,JFK,22,16321,ZRH,486,19790,DL,1,11074,0,11074
67501,2022-06-01 00:00:00,12478,JFK,22,16321,ZRH,486,20233,LX,0,12825,0,12825
67502,2022-06-01 00:00:00,12478,JFK,22,16321,ZRH,486,21202,CAZ,0,0,1,1
67503,2022-06-01 00:00:00,12478,JFK,22,16782,DSS,569,19790,DL,1,5537,0,5537


### Identifing all airlines whose name begins with Air

In [9]:
df = pd.read_sql("select airline_name from airline_codes where airline_name like 'Air%'",conn)
df.head(10)

,airline_name
0,Air America Inc.
1,Air Micronesia Inc.
2,Air Cargo Express Inc.
3,Air Bahia
4,Airbama Incorporated
5,Air Cargo Enterprises Inc.
6,Air Chaparral Inc.
7,Air Central Inc. (1)
8,Air Central Inc.
9,Air Cargo America Inc.


### Identifying flights flown between May 2019 and June 2019

In [31]:
df = pd.read_sql("SELECT *\
                 FROM passenger_travel_data\
                 WHERE strftime('%Y-%m-%d', data_dte) BETWEEN '2019-05-01' AND '2019-06-01'\
                 ",conn)
df

,data_dte,usg_apt_id,usg_apt,usg_wac,fg_apt_id,fg_apt,fg_wac,airlineid,carrier,carriergroup,scheduled,charter,total
0,2019-05-01 00:00:00,10136,ABI,74,13514,MTY,148,20398,MQ,1,51,0,51
1,2019-05-01 00:00:00,10136,ABI,74,16257,YYC,916,20398,MQ,1,68,0,68
2,2019-05-01 00:00:00,10140,ABQ,86,11032,CUN,148,19393,WN,1,138,0,138
3,2019-05-01 00:00:00,10140,ABQ,86,11874,GDL,148,21351,Y4,0,244,0,244
4,2019-05-01 00:00:00,10154,ACK,13,16271,YYZ,936,21437,13Q,0,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5359,2019-06-01 00:00:00,99999,ZZZ,1,14682,SAQ,204,21743,2LQ,1,0,13,13
5360,2019-06-01 00:00:00,99999,ZZZ,1,16233,YWH,906,20272,KAH,1,0,2,2
5361,2019-06-01 00:00:00,99999,ZZZ,1,16310,ZNA,906,20272,KAH,1,0,2,2
5362,2019-06-01 00:00:00,99999,ZZZ,1,16599,YBW,906,20272,KAH,1,0,6,6


### Most Common Takeoff Place in USA

In [15]:
df = pd.read_sql("\
                select ptd.usg_apt, ac.airport_name,count(ptd.usg_apt) as airport_count\
                from passenger_travel_data ptd\
                join airport_codes ac on ptd.usg_apt == ac.airport_code\
                group by ptd.usg_apt,ac.airport_name\
                order by count(ptd.usg_apt) desc\
                limit 10",conn)
df

,usg_apt,airport_name,airport_count
0,JFK,John F Kennedy Intl,67505
1,MIA,Miami International Airport,65435
2,LAX,Los Angeles International,50791
3,ORD,Ohare International,37034
4,EWR,Newark International Airport,35087
5,IAH,George Bush Intercntl.,29089
6,ATL,Hartsfield-Jackson Atlanta Int,25998
7,SFO,San Francisco International,22082
8,BOS,Logan International,21265
9,DFW,Dallas/Ft Worth Intl,20576


### To which country do the passengers travel the most

In [16]:
df = pd.read_sql("\
                select ac.country_name,count(ptd.fg_apt) as country_count\
                from passenger_travel_data ptd\
                join airport_codes ac on ptd.fg_apt == ac.airport_code\
                group by ac.country_name\
                order by count(ptd.fg_apt) desc\
                limit 10",conn)
df

,country_name,country_count
0,Canada,136471
1,Mexico,126667
2,United Kingdom,41852
3,Japan,31593
4,Bahamas,26365
5,Germany,25021
6,Dominican Republic,22243
7,France,17356
8,Jamaica,15336
9,Brazil,13102


### Most Used Airlines by the Passengers

In [17]:
df = pd.read_sql("\
                 select ptd.carrier, ac.airline_name,count(ptd.scheduled) as airline_count\
                 from passenger_travel_data ptd\
                 join airline_codes ac on ptd.carrier == ac.airline_id\
                 group by ptd.carrier,ac.airline_name\
                 order by count(ptd.scheduled) desc\
                 limit 10",conn)
df

,carrier,airline_name,airline_count
0,AA,American Airlines Inc.,63012
1,DL,Delta Air Lines Inc.,50984
2,UA,United Air Lines Inc.,46433
3,CO,Continental Air Lines Inc.,25533
4,AC,Air Canada,21001
5,NW,Northwest Airlines Inc.,13326
6,AM,Aeromexico,10892
7,MX,Breeze Aviation Group DBA Breeze,10757
8,BA,British Airways Plc,10692
9,B6,JetBlue Airways,10547


### Average number of Passengers flown by a airline in a month

In [21]:
df = pd.read_sql("\
                 select ptd.carrier, ac.airline_name, avg(ptd.total)\
                 from passenger_travel_data ptd\
                 join airline_codes ac on ptd.carrier == ac.airline_id\
                 group by ptd.carrier, ac.airline_name\
                 order by avg(ptd.total) desc\
                 limit 10",conn)
df

,carrier,airline_name,avg(ptd.total)
0,EK,Emirates,20335.086526
1,VS,Virgin Atlantic Airways,20027.389395
2,S3,"Santa Barbara Airlines, C.A",18554.321429
3,BR,Eva Airways Corporation,17143.827442
4,CX,Cathay Pacific Airways Ltd.,16938.212981
5,BA,British Airways Plc,16589.239712
6,JO,Jalways Co. Ltd.,15936.311535
7,QR,Qatar Airways (Q.C.S.C),14704.352632
8,AF,Compagnie Natl Air France,14628.271011
9,KL,Klm Royal Dutch Airlines,14132.110775


### Ranking top places most passengers visited in 2022

In [32]:
df = pd.read_sql("\
                 select ptd.fg_apt,ac.airport_name, ac.country_name,rank() over (order by avg(ptd.total) desc) rank\
                 from passenger_travel_data ptd\
                 join airport_codes ac on ptd.fg_apt == ac.airport_code\
                 where strftime('%Y', data_dte) == '2022'\
                 group by ptd.fg_apt,ac.airport_code, ac.country_name\
                 order by avg(ptd.total) desc\
                 limit 10",conn)
df

,fg_apt,airport_name,country_name,rank
0,IST,Ataturk,Turkey,1
1,DOH,Doha,Qatar,2
2,DXB,Dubai,United Arab Emirates,3
3,LHR,Heathrow,United Kingdom,4
4,CDG,Charles De Gaulle,France,5
5,STI,Municipal,Dominican Republic,6
6,FCO,Leonardo Da Vinci (Fiumicino),Italy,7
7,AMS,Amsterdam-Schiphol,Netherlands,8
8,GRU,Guarulhos Intl,Brazil,9
9,EZE,Ezeiza Ministro Pistarini,Argentina,10


### Finding the percente change in travel during the year 2020 

In [29]:
df = pd.read_sql("\
                 select month,\
                 (total_passenger - LAG(total_passenger) OVER (ORDER BY month)) * 100.0 / total_passenger AS percent_change\
                 from (\
                     select strftime('%m', data_dte) as month, sum(ptd.total) as total_passenger\
                     from passenger_travel_data ptd\
                     where strftime('%Y', data_dte) == '2020'\
                     group by strftime('%m', data_dte)\
                 )\
                 ",conn)
df

,month,percent_change
0,01,NaN
1,02,-19.395989
2,03,-76.432968
3,04,-1989.788209
4,05,-9.716281
5,06,52.143026
6,07,56.133583
7,08,18.660259
8,09,-0.516093
9,10,22.749524


### Most Common Airline going to France from US 

In [33]:
df = pd.read_sql("\
                 select ptd.carrier,alc.airline_name, count(ptd.carrier)\
                 from passenger_travel_data ptd\
                 join airport_codes ac on ptd.fg_apt == ac.airport_code\
                 join airline_codes alc on ptd.carrier == alc.airline_id\
                 where ac.country_name == 'France'\
                 group by ptd.carrier,alc.airline_name\
                 order by count(ptd.carrier) DESC\
                 limit 10\
                 ",conn)
df

,carrier,airline_name,count(ptd.carrier)
0,AF,Compagnie Natl Air France,4635
1,DL,Delta Air Lines Inc.,2565
2,AA,American Airlines Inc.,2027
3,UA,United Air Lines Inc.,1300
4,VJT,VistaJet Limited,611
5,CO,Continental Air Lines Inc.,601
6,TW,Trans World Airways LLC,514
7,NW,Northwest Airlines Inc.,315
8,FF,Tower Air Inc.,255
9,DY,Norwegian Air Shuttle ASA,247


### Identifying countries passengers have flown from a non-US carrier airline in 2018 except Canada and Mexico 

In [38]:
df = pd.read_sql("\
    select ac.country_name,count(ptd.fg_apt) as count\
    from passenger_travel_data as ptd\
    join airport_codes ac on ptd.fg_apt == ac.airport_code\
    where (cast(substr(data_dte,1,4) as integer) = 2018) and ptd.carriergroup = 0 and ac.country_name not in ('Canada','Mexico')\
    group by ac.country_name\
    order by count(ptd.fg_apt) desc\
    limit 10",conn)

df

,country_name,count
0,United Kingdom,1271
1,China,675
2,Germany,592
3,France,527
4,Japan,486
5,Iceland,340
6,South Korea,325
7,Ireland,302
8,Brazil,265
9,Spain,255
